In [1]:
import make_dataset as md
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.cluster.vq import vq, kmeans
from scipy.spatial.distance import cdist
import pandas as pd
import cv2

In [2]:
# Get Image Descriptors, which are a combination of points on an image and the description of surrounding pixels.

def get_descriptors(nparrays,nfeatures):
    sift = cv2.SIFT_create(nfeatures = nfeatures)
    container = []
    for i in nparrays:
        img_bw = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
        keypoint,descriptor = sift.detectAndCompute(img_bw,None)
        container.append([keypoint,descriptor])
    return container

#For a collection of image area descriptions, get the Kmeans of n clusters. This will be what future images are compared to.

def get_vocab(descriptors,n):
    descriptor_container = []
    for i in descriptors:
        if i[1] is None:
            continue
        for j in i[1]:
            descriptor_container.append(j)
    vocab = kmeans(descriptor_container,n)
    return vocab

# For each image, get its descriptors. For each descriptor, get the closest Kmean descriptor in vocab and add 1 to its index in a histogram.
# Return a histogram per image. This histogram will be passed as a feature for modeling.

def descriptor_to_vocab(nparrays,vocab):
    sift = cv2.SIFT_create()
    container = []
    for i in nparrays:
        img_bw = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
        keypoint,descriptor = sift.detectAndCompute(img_bw,None)
        if descriptor is not None:
            dist = cdist(descriptor,vocab[0],'euclidean')
            bin_assignment = np.argmin(dist,axis = 1)
        else:
            bin_assignment = []
        image_feats = np.zeros(len(vocab[0]))
        for j in bin_assignment:
            image_feats[j] += 1
        container.append(image_feats)
    return container

# Normalizes histograms from images so that they may be used in ML inputs

def normalize_histograms(histarray):
    histarray = np.array(histarray)
    feats_norm_div = np.linalg.norm(histarray,axis = 1)
    for i in range(0,histarray.shape[0]):
        histarray[i] = histarray[i]/feats_norm_div[i]
    return histarray

# Pipeline for SIFT to histogram features per image
'''
path - path to the image directory
column - what type of label to return
nvocab - how many vectors will appear in the descriptor 'vocabulary'
test_size - percentage of images that will be used as tests
random_state - seed for randomization
n_features - For each image, return at most n descriptors
'''

def SIFT_to_Features(path,column,nvocab = 200,test_size = 0.33,random_state = 42,n_features = 100):
    filedf = md.get_image_label_filepath_df('./Data/CroppedImages/')
    X,y = md.get_labels_npimages(filedf,column)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    X_train_np = np.array(X_train)
    X_test_np = np.array(X_test)
    y_train_np = np.array(y_train)
    y_test_np = np.array(y_test)
    descriptors = get_descriptors(X_train_np[:,1],n_features)
    vocab = get_vocab(descriptors[:100],nvocab)
    # Get histogram values from X_train descriptors
    histarraytrain = descriptor_to_vocab(X_train_np[:,1],vocab)
    # Get histogram values from X_test descriptors
    histarraytest = descriptor_to_vocab(X_test_np[:,1],vocab)
    # Normalize train histograms
    normalizehisttrain = normalize_histograms(histarraytrain)
    # Normalize test histograms
    normalizehisttest = normalize_histograms(histarraytest)
    return [(X_train_np[:,0],normalizehisttrain,y_train_np),(X_test_np[:,0],normalizehisttest,y_test_np)]


In [11]:
%%time
train,test = SIFT_to_Features('./Data/CroppedImages/','race')

C:\Users\thanh\AppData\Local\Temp\ipykernel_10160\3187525907.py:56: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_np = np.array(X_train)
C:\Users\thanh\AppData\Local\Temp\ipykernel_10160\3187525907.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_np = np.array(X_test)
C:\Users\thanh\AppData\Local\Temp\ipykernel_10160\3187525907.py:49: RuntimeWarning: invalid value encountered in true_divide
  histarray[i] = histarray[i]/feats_norm_div[i]


CPU times: total: 18min 14s
Wall time: 4min 25s


In [12]:
traindf = pd.DataFrame(train)
testdf = pd.DataFrame(test)

In [13]:
traindf = traindf.transpose()
testdf = testdf.transpose()

In [14]:
traindf.to_csv('filename',index=False)
testdf.to_csv('filename',index=False)